In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from pathlib import Path
from pystackreg import StackReg
from scipy.ndimage import gaussian_filter

import utils as utl

# User settings

In [ ]:
# number of channels and z slices
n_ch, n_z = 2, 15
# smoothing in xy before registration (sigma 2D Gaussian)
xy_smth = 4

# frequencies used for imaging, ball velocities, and behavior
f_ca, f_ball, f_beh = 2, 50, 200

# transformation used for registration https://pystackreg.readthedocs.io/en/latest/readme.html#usage
reg = StackReg.SCALED_ROTATION 

# path to folder
parent_dir = Path(r'\\mpfi.org\public\sb-lab\Nino_2P_for_Salil\for_Nico\stop1_imaging\stop1-GCaMP6f-tdTomato_VNC_smth8')

# selection rule for tif files
p_tifs = [ *parent_dir.glob('**/trials_to_register/*/trial*_00???.tif') ]

# force overwriting files
overwrite = True

# folder to save output
p_out = parent_dir / 'pipeline_output'
p_out.mkdir(exist_ok=True)
p_all = p_out / 'all_data.parquet'

# general processing pipeline
## Step 1: Registration

In [ ]:
for p_tif in p_tifs:
    print()

    if utl.fname(p_tif, 'ch1.tif').is_file() and not overwrite:
        print(f'INFO output file exists, skipping registration for {p_tif.parent}')
        continue
    else:
        print(f'INFO now registering {p_tif}')

    # load and split
    stack = utl.load_tiff(p_tif)
    ch1, ch2 = utl.split_channels(stack, n_z=15, n_ch=2)
    ch1 = utl.maxproj_z(ch1)
    ch2 = utl.maxproj_z(ch2)

    ch1 = utl.smooth_xy(ch1, xy_smth)
    ch2 = utl.smooth_xy(ch2, xy_smth)

    # register
    tmats = utl.get_tmats(ch2, reg)
    ch1_a = utl.align(ch1, tmats, reg)
    ch2_a = utl.align(ch2, tmats, reg)

    # mean image
    ch1_am = np.mean(ch1_a, axis=0)
    ch2_am = np.mean(ch2_a, axis=0)

    # save to disk
    utl.write_tif(utl.fname(p_tif, 'ch1.tif'), ch1_a.astype('int16'))
    utl.write_tif(utl.fname(p_tif, 'ch2.tif'), ch2_a.astype('int16'))

    utl.write_tif(utl.fname(p_tif, 'ch1reg.tif'), ch1_a.astype('int16'))
    utl.write_tif(utl.fname(p_tif, 'ch2reg.tif'), ch2_a.astype('int16'))

    utl.save_img(utl.fname(p_tif, 'ch1mean.bmp'), ch1_am)
    utl.save_img(utl.fname(p_tif, 'ch2mean.bmp'), ch2_am)

    utl.save_dual_movie(utl.fname(p_tif, 'ch1ch2.mp4'), ch1, ch2)
    utl.save_dual_movie(utl.fname(p_tif, 'ch1reg.mp4'), ch1, ch1_a)
    utl.save_dual_movie(utl.fname(p_tif, 'ch2reg.mp4'), ch2, ch2_a)
    


## Step 2: ROI extraction

In [ ]:
for p_tif in p_tifs:
    print()
    
    # check if ROI traces have already been extracted
    p_roi = utl.fname(p_tif, 'roi_traces.npy')
    if p_roi.is_file() and not overwrite:
        print(f'INFO output files exists, skipping ROI extraction for {p_tif.parent}')
        continue

    # load Roi.zip
    p_zip = utl.get_roi_zip_file(p_tif)
    if not p_zip:
        print(f'WARNING Skipping {p_tif.parent}')
        continue
    else:
        print(f'INFO loading ROIs from {p_zip}')

    # load aligned ch1
    stack = utl.load_tiff(utl.fname(p_tif, 'ch1reg.tif'))
    # stack = utl.load_tiff(utl.fname(p_tif, 'ch2reg.tif'))
    img = np.mean(stack, axis=0)

    # load ROIs
    rois = utl.read_imagej_rois(p_zip, img)
    img_rois = utl.draw_rois(img, rois)
    utl.save_img(utl.fname(p_tif, 'ch1mean_rois.bmp'), img_rois)

    # extract traces
    ca = utl.get_mean_trace(rois, stack, subtract_background=True, sigma=0)
    np.save(p_roi, ca)    
    print(f'INFO saving ROI traces to {p_roi}')  


## Step 3: Combine imaging and behavior data

In [ ]:
for p_tif in p_tifs:
    print()
    
    # check if already been processed
    p_df = utl.fname(p_tif, 'data.parquet')
    if p_df.is_file() and not overwrite:
        print(f'INFO output files exists, skipping data merging for {p_tif.parent}')
        continue

    # load ROI traces
    p_roi = utl.fname(p_tif, 'roi_traces.npy')
    if not p_roi.is_file():
        print(f'WARNING file with ROI traces not found, skipping {p_tif.parent}')
    else:
        ca = np.load(p_roi)

    # load behavior data and ball velocities
    p_mats = utl.get_matlab_files(p_tif)
    if not p_mats:
        print(f'WARNING skipping {p_tif.parent}')
    else:
        p_ball, p_beh =  p_mats
        
    ball = utl.load_ball(p_ball)
    beh = utl.load_behavior(p_beh)

    # match sample rates
    df = utl.upsample_to_behavior(ca, beh, ball, f_ca, f_ball, f_beh)
    # zscore ROIs
    df = utl.zscore_cols(df, col_start='roi_')
    # convolute ball velocities and behavior with Ca kernel
    df = utl.convolute_ca_kernel(df, f=f_beh)
    # zscore ball velocities
    df = utl.zscore_cols(df, col_start='conv_ball_')

    # add additional data based on file and folder names
    pt = p_tif.parts
    cond, fly, trial = pt[-5], pt[-4], pt[-2]
    df.loc[:, 'cond'] = cond # e.g. fed/starved
    df.loc[:, 'fly'] = fly # fly number
    df.loc[:, 'trial'] = trial # trial number
    print(f'INFO parsing folder names: fly {fly} | trial {trial} | condition {cond}')

    # plots for quality control
    utl.plot_data(df, f_beh, path=utl.fname(p_tif, 'data.png'))
    # pearson r heatmap
    utl.plot_corr_heatmap(df, beh='behi', path=utl.fname(p_tif, 'heatmap.png'))
    # ccf
    utl.plot_ccf(df, f=f_beh, pool_fly=True, path=utl.fname(p_tif, 'ccf.png'))

    # save to disk
    print(f'INFO writing merged data to {p_df}')
    df.to_parquet(p_df)

## Step 4: merge all trials

In [ ]:
# merge all trials and flies

# list of all *_data.parquet files
p_pars = [ utl.fname(p, 'data.parquet') for p in p_tifs ]

l = []
for p_par in p_pars:
    print()
    if not p_par.is_file():
        print(f'WARNING skipping {p_par.parent}')
        continue
    else:
        print(f'INFO loading file {p_par}')
        df = pd.read_parquet(p_par)
        l.append(df)

# combine dataframes and save
df = pd.concat(l, ignore_index=True)
df.to_parquet(p_all)

print(f'INFO contents of {p_all}')
for f, d in df.groupby('fly'):
    print(f'     {f}', end=': ')
    for t, _ in d.groupby('trial'):
        print(f'{t}', end=' ')
    print()

# Analysis

## All recordings

In [ ]:
# read data from disk
df = pd.read_parquet(p_all)
df = df.fillna(0) # TODO workaround because of missing behavior

# optional: remove ROI 7, 8, 9
df = df.drop(columns=['z_roi_7', 'z_roi_8', 'z_roi_9'])

### Pearson's R heatmaps

In [ ]:
# pearson correlation heatmap (selection of columns: see utl.calculate_pearson)
utl.plot_corr_heatmap(df, beh='behi', path=p_out / 'heatmap.svg')

In [ ]:
# pearson heatmaps around behavior events
dt = 5 # time in s to keep before and after behavior event

# loop through all behavior
cols = [ c for c in df.columns if c.startswith('beh_') ]
for col in cols:

    # select df around behavoir
    d = utl.select_event(df, col, f_beh, dt)

    # generate plot
    utl.plot_corr_heatmap(d, beh='behi', path=p_out / f'heatmap_{col}.svg')

### behavior/ball and ROI cross-correlation

In [ ]:
# cross-correlation functions behavior/ball and ROIs (averaged)
utl.plot_ccf(df, f=f_beh, pool_fly=True,  path=p_out / 'ccf.svg')

# same (not averaged)
utl.plot_ccf(df, f=f_beh, pool_fly=False, path=p_out / 'ccf_indv.svg')

### average ROIs and ball velocities aligned to behavior

In [ ]:
dt = 5 # time in s before and after behavior event
s = 0.25 # smoothing window for velocity [in s] 

# smooth velocity
df_ = df.copy()
df_.loc[:, ['ball_x', 'ball_y', 'ball_z']] = gaussian_filter(df_.loc[:, ['ball_x', 'ball_y', 'ball_z']].values, (s * f_beh, 0))

# cycle through all behavoirs
cols = [c for c in df_.columns if c.startswith('beh_')]
for col in cols:

    # align to behavior
    df_al = utl.align2events(df_, col, f_beh, dt)

    utl.plot_aligned(df_al, path=p_out / f'aligned_to_{col}.svg')

## individual recording

In [ ]:
# define tif file for session of interest
p_tifs = [ *Path(r'Y:\Nino_2P_for_Salil\for_Nico\stop1_imaging\stop1-GCaMP6f-tdTomato_VNC\fed\female13\trials_to_register').glob('**/*_000??.tif') ]
p_tif = p_tifs[2]

### correlation map

In [ ]:
# define and create output folder
p_out = p_tif.parent / 'corrmap'
p_out.mkdir(exist_ok=True)

# load registered tif files and smooth x/y
ch1 = utl.load_tiff(utl.fname(p_tif, 'ch1reg.tif'))
ch2 = utl.load_tiff(utl.fname(p_tif, 'ch2reg.tif'))
arr1 = gaussian_filter(ch1, sigma=(0, 1, 1))
arr2 = gaussian_filter(ch2, sigma=(0, 1, 1))

# load preprocessed behavior data
df = pd.read_parquet(utl.fname(p_tif, 'data.parquet'))

# loop through all conv behi and conv ball columns
cols = [ c for c in df.columns if c.startswith('conv_behi') or c.startswith('conv_ball') ]
for col in cols:
    utl.plot_corrmap(arr1, arr2, df, col, f_ca=f_ca, f_beh=f_beh, path=p_out / f'{col}_1xy.svg')